# Time series analysis with HyP3 and MintPy

This notebook walks through performing a time-series analysis of the 2019 Ridgecrest, CA earthquake with On Demand InSAR products from the Alaska Satellite facility and MintPy. We'll:

1. Use the [ASF Search Python package](https://docs.asf.alaska.edu/asf_search/basics/) to:
   - Search ASF's catalog for Sentinel-1 SAR products covering the [Ridgecrest](https://earthquake.usgs.gov/storymap/index-ridgecrest.html)
   - Select a reference scene to generate a baseline stack
   - Select a [short baseline subset (SBAS)](https://docs.asf.alaska.edu/vertex/sbas/) of scene pairs for InSAR processing


2. Use the [HyP3 Python SDK](https://hyp3-docs.asf.alaska.edu/using/sdk/) to:
   - Request On Demand InSAR products from ASF HyP3
   - Download the InSAR products when they are done processing


3. Use [GDAL](https://gdal.org/api/index.html#python-api) and [MintPy](https://mintpy.readthedocs.io/en/latest/) to:
   - Prepare the InSAR products for MintPy
   - perform a time-series analysis with MintPy
   
---

**Note:** This notebook does assume you have some familiarity with InSAR processing with MintPy already, and is a minimal example without much context or explanations. If you're new to InSAR and MintPy, I suggest checking out:
* our [InSAR on Demand story map](https://storymaps.arcgis.com/stories/68a8a3253900411185ae9eb6bb5283d3)


* [OpenSARlab's](https://opensarlab-docs.asf.alaska.edu/) highly detailed walkthrough of using HyP3 + MintPy via these notebooks:
  * [Prepare a HyP3 InSAR Stack for MintPy](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb)
  * [MintPy Time-series Analysis](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/MintPy_Time_Series_From_Prepared_Data_Stack.ipynb)
  
    Note: While these notebooks make some assumptions you're working in OpenSARlab, you can run these 
    notebooks outside OpenSARlab by creating [this conda environment](https://github.com/ASFOpenSARlab/opensarlab-envs/blob/main/Environment_Configs/insar_analysis_env.yml).

## 0. Initial Setup

To run this notebook, you'll need a conda environment with the required dependencies. You can set up a new environment (recommended) and run the jupyter server like:
```shell
conda create -n hyp3-mintpy python=3.8 asf_search hyp3_sdk "mintpy>=1.3.2" pandas jupyter ipympl

conda activate hyp3-mintpy
jupyter notebook hyp3_insar_stack_for_ts_analysis.ipynb
```
Or, install these dependencies into your own environment:
```shell
conda install hyp3-mintpy python=3.8 asf_search hyp3_sdk "mintpy>=1.3.2" pandas jupyter ipympl

jupyter notebook hyp3_insar_stack_for_ts_analysis.ipynb
```

In [ ]:

import os

from pathlib import Path

from dateutil.parser import parse as parse_date


### set parameters

In [ ]:

project_name = '2019_ridgecrest'
project_home = Path.cwd()
work_dir = project_home / project_name
data_dir = work_dir / 'data'

stack_start = parse_date('2019-06-10')
stack_end = parse_date('2019-07-21')
max_temporal_baseline = 13 #days

data_dir.mkdir(parents=True, exist_ok=True)

os.chdir(work_dir)

print('Go to work directory: {}'.format(work_dir))


## 1. Select InSAR pairs with ASF Search

In [ ]:

import asf_search as asf
import pandas as pd

intersect_point = (-117.599330, 35.769500)

date_range=('2019-06-10', '2019-07-21')

search_results = asf.geo_search(
        platform=asf.SENTINEL1,
        intersectsWith=f'POINT({intersect_point[0]} {intersect_point[1]})',
        start=date_range[0],
        end=date_range[1],
        processingLevel=asf.SLC,
        beamMode=asf.IW,
        flightDirection=asf.ASCENDING,
    )


In [ ]:

baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]


In [ ]:

sbas_pairs = set()

for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack.loc[
        (stack.sceneName != reference)
        & (stack.temporalBaseline - rt <= max_temporal_baseline)
        & (stack.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))


## 2. Request On Demand InSAR products from ASF HyP3

Use your [NASA Earthdata login](https://urs.earthdata.nasa.gov/) to connect to [ASF HyP3](https://hyp3-docs.asf.alaska.edu/).

In [ ]:

import hyp3_sdk as sdk

hyp3 = sdk.HyP3(prompt=True)


In [ ]:

jobs = sdk.Batch()

for reference, secondary in sbas_pairs:
    jobs += hyp3.submit_insar_job(reference, secondary, name=project_name,
                                  include_dem=True, include_look_vectors=True)


In [ ]:

jobs = hyp3.find_jobs(name=project_name)


In [ ]:

jobs = hyp3.watch(jobs)


In [ ]:

insar_products = jobs.download_files(data_dir)

insar_products = [sdk.util.extract_zipped_product(ii) for ii in insar_products]


## 3. Time-series Analysis with MintPy

### 3.1 cut off the geotiff files

In [ ]:

from osgeo import gdal

def get_overlap(filelist):
    '''
    :param list of geotif files.
    :process get the smallest overlap retangular area of the geotiff files.
    :return: overlap area [ulx, uly, lrx, lry]
    '''
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in filelist]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]


In [ ]:

import glob

filelist = glob.glob(f"{data_dir}/*/*_dem.tif")

overlap = get_overlap(filelist)


In [ ]:

def prepare_hyp3_product(data_dir, overlap):

    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

    list_product_dirs = [f.path for f in os.scandir(data_dir) if f.is_dir()]

    for product_dir in list_product_dirs:
        for file_suffix in files_for_mintpy:
            product_dir = Path(product_dir)
            src_file = product_dir / f'{product_dir.name}{file_suffix}'
            dst_file = product_dir / f'{src_file.stem}_clipped{src_file.suffix}'
            gdal.Translate(destName=str(dst_file), srcDS=str(src_file), projWin=overlap)


In [ ]:

prepare_hyp3_product(data_dir, overlap)


### 3.2 create the config file

In [ ]:

mintpy_config = work_dir / 'mintpy_config.txt'
mintpy_config.write_text(
f"""
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {data_dir}/*/*_unw_phase_clipped.tif
mintpy.load.corFile          = {data_dir}/*/*_corr_clipped.tif
##---------geometry datasets:
mintpy.load.demFile          = {data_dir}/*/*_dem_clipped.tif
mintpy.load.incAngleFile     = {data_dir}/*/*_lv_theta_clipped.tif
mintpy.load.azAngleFile      = {data_dir}/*/*_lv_phi_clipped.tif
mintpy.load.waterMaskFile    = {data_dir}/*/*_water_mask_clipped.tif
""")


### 3.3 run mintpy to do time series analysis

In [ ]:

!smallbaselineApp.py --dir {work_dir} {mintpy_config}


In [ ]:

%matplotlib widget
from mintpy import view, tsview


In [ ]:

view.main([f'{work_dir}/velocity.h5'])


In [ ]:

tsview.main([f'{work_dir}/timeseries.h5'])
